# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [40]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [22]:
df = pd.read_csv('previsao_de_renda.csv')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [24]:
df = df.dropna(axis=0)
df.sexo=df.sexo.map({'F': 1, 'M':0})
df= pd.get_dummies(df, columns=['tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia'],drop_first=True)
df = df.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)
df.dtypes

Unnamed: 0                         int64
data_ref                          object
id_cliente                         int64
sexo                               int64
posse_de_veiculo                   int32
posse_de_imovel                    int32
qtd_filhos                         int64
idade                              int64
tempo_emprego                    float64
qt_pessoas_residencia            float64
renda                            float64
tipo_renda_Bolsista                int32
tipo_renda_Empresário              int32
tipo_renda_Pensionista             int32
tipo_renda_Servidor público        int32
educacao_Pós graduação             int32
educacao_Secundário                int32
educacao_Superior completo         int32
educacao_Superior incompleto       int32
estado_civil_Separado              int32
estado_civil_Solteiro              int32
estado_civil_União                 int32
estado_civil_Viúvo                 int32
tipo_residencia_Casa               int32
tipo_residencia_

In [25]:
df.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor público', 'educacao_Pós graduação',
       'educacao_Secundário', 'educacao_Superior completo',
       'educacao_Superior incompleto', 'estado_civil_Separado',
       'estado_civil_Solteiro', 'estado_civil_União', 'estado_civil_Viúvo',
       'tipo_residencia_Casa', 'tipo_residencia_Com os pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental'],
      dtype='object')

In [26]:
df.columns=['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia','renda', 'tipo_renda_Bolsista', 'tipo_renda_Empresário',
       'tipo_renda_Pensionista', 'tipo_renda_Servidor_público',
       'educacao_Pós_graduação', 'educacao_Secundário',
       'educacao_Superior_completo', 'educacao_Superior_incompleto',
       'estado_civil_Separado', 'estado_civil_Solteiro', 'estado_civil_União',
       'estado_civil_Viúvo', 'tipo_residencia_Casa',
       'tipo_residencia_Com_os_pais', 'tipo_residencia_Comunitário',
       'tipo_residencia_Estúdio', 'tipo_residencia_Governamental']
df.columns

Index(['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo',
       'posse_de_imovel', 'qtd_filhos', 'idade', 'tempo_emprego',
       'qt_pessoas_residencia', 'renda', 'tipo_renda_Bolsista',
       'tipo_renda_Empresário', 'tipo_renda_Pensionista',
       'tipo_renda_Servidor_público', 'educacao_Pós_graduação',
       'educacao_Secundário', 'educacao_Superior_completo',
       'educacao_Superior_incompleto', 'estado_civil_Separado',
       'estado_civil_Solteiro', 'estado_civil_União', 'estado_civil_Viúvo',
       'tipo_residencia_Casa', 'tipo_residencia_Com_os_pais',
       'tipo_residencia_Comunitário', 'tipo_residencia_Estúdio',
       'tipo_residencia_Governamental'],
      dtype='object')

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [27]:
X=df.drop(columns=['renda','data_ref'])
y=df['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)
df_train=pd.concat([X_train,y_train],axis=1)

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [28]:
alpha=[0, 0.001, 0.005, 0.01, 0.05, 0.1]
sumarios=[]
for i in alpha:
    modelo = '''renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tipo_renda_Bolsista 
            + tipo_renda_Empresário + tipo_renda_Pensionista + tipo_renda_Servidor_público + educacao_Pós_graduação
            + educacao_Secundário + educacao_Superior_completo + educacao_Superior_incompleto
            + estado_civil_Separado + estado_civil_Solteiro + estado_civil_União + estado_civil_Viúvo
            + tipo_residencia_Casa + tipo_residencia_Com_os_pais + tipo_residencia_Comunitário
            + tipo_residencia_Estúdio + tipo_residencia_Governamental + tempo_emprego + qt_pessoas_residencia'''
    md = smf.ols(modelo, data = df_train)
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 0.01
                             , alpha = i)
    sumarios.append(reg.summary())
    print(f'No alpha {i} ')
    print(sumarios)


No alpha 0 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:54   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

No alpha 0.005 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:54   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

No alpha 0.01 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:54   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

No alpha 0.05 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:54   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

No alpha 0.1 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:54   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

Todos os valores de $R^2$ foram iguais

3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [29]:
alpha=[0, 0.001, 0.005, 0.01, 0.05, 0.1]
sumarios=[]
for i in alpha:
    modelo = '''renda ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tipo_renda_Bolsista 
            + tipo_renda_Empresário + tipo_renda_Pensionista + tipo_renda_Servidor_público + educacao_Pós_graduação
            + educacao_Secundário + educacao_Superior_completo + educacao_Superior_incompleto
            + estado_civil_Separado + estado_civil_Solteiro + estado_civil_União + estado_civil_Viúvo
            + tipo_residencia_Casa + tipo_residencia_Com_os_pais + tipo_residencia_Comunitário
            + tipo_residencia_Estúdio + tipo_residencia_Governamental + tempo_emprego + qt_pessoas_residencia'''
    md = smf.ols(modelo, data = df_train)
    reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 1
                             , alpha = i)
    sumarios.append(reg.summary())
    print(f'No alpha {i} ')
    print(sumarios)

No alpha 0 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:58   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

No alpha 0.005 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:58   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------

No alpha 0.01 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:58   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

No alpha 0.05 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:58   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------

No alpha 0.1 
[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.247
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     121.7
Date:                Wed, 08 May 2024   Prob (F-statistic):               0.00
Time:                        20:41:58   Log-Likelihood:                -96962.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9295   BIC:                         1.942e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

Todos os valores de $R^2$ foram iguais

4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [30]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X, y)

print('resulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 0.0
#############
['tempo_emprego']
Add  sexo                           with p-value 0.0
#############
['tempo_emprego', 'sexo']
Add  tipo_renda_Empresário          with p-value 1.75299e-07
#############
['tempo_emprego', 'sexo', 'tipo_renda_Empresário']
Add  idade                          with p-value 1.9605e-07
#############
['tempo_emprego', 'sexo', 'tipo_renda_Empresário', 'idade']
Add  educacao_Superior_completo     with p-value 3.07164e-06
#############
['tempo_emprego', 'sexo', 'tipo_renda_Empresário', 'idade', 'educacao_Superior_completo']
Add  qt_pessoas_residencia          with p-value 0.00747762
#############
['tempo_emprego', 'sexo', 'tipo_renda_Empresário', 'idade', 'educacao_Superior_completo', 'qt_pessoas_residencia']
Add  posse_de_imovel                with p-value 0.0138522
#############
['tempo_emprego', 'sexo', 'tipo_renda_Empresário', 'idade', 'educacao_Superior_completo', 'qt_pessoas_residencia', 'posse_de_imovel']
#

In [36]:
modelo = '''renda ~ tempo_emprego + sexo + tipo_renda_Empresário + idade + educacao_Superior_completo 
            + qt_pessoas_residencia + posse_de_imovel'''
md = smf.ols(modelo, data = df_train)
reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 1
                             , alpha = 0.05)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.246
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     379.4
Date:                Mon, 13 May 2024   Prob (F-statistic):               0.00
Time:                        22:18:07   Log-Likelihood:                -96967.
No. Observations:                9320   AIC:                         1.940e+05
Df Residuals:                    9312   BIC:                         1.940e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   3022.6835    501.791      6.024      0.000    2039.064    4006.303
tempo_emprego                564.9946     13.112     43.091      0.000     539.293     590.696
sexo                       -6166.0116    175.474    -35.139      0.000   -6509.978   -5822.045
tipo_renda_Empresário        749.0822    185.938      4.029      0.000     384.602    1113.562
idade                         45.7441      9.694      4.719      0.000      26.741      64.747
educacao_Superior_completo   446.4101    172.095      2.594      0.010     109.066     783.755
qt_pessoas_residencia         90.9492     90.926      1.000      0.317     -87.287     269.185
posse_de_imovel              297.2578    174.208      1.706      0.088     -44.228     638.744
==============================================================================
Omnibus:                    13670.210   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          8620824.862
Skew:                           8.697   Prob(JB):                         0.00
Kurtosis:                     150.976   Cond. No.                         262.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

O resultados são muitos próximos com stepwise $R^2$ de 2,46 sem de 2,47 com $R^2$ adjustado de 0,245 em ambos.

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

O resultados são muitos próximos com stepwise $R^2$ de 2,46 sem de 2,47 com $R^2$ adjustado de 0,245 em ambos. Acredito que o modelo com stepwise ser melhor por ser menos parâmetros.

6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.


In [37]:
modelo = '''np.log(renda) ~ tempo_emprego + sexo + tipo_renda_Empresário + idade + educacao_Superior_completo 
            + qt_pessoas_residencia + posse_de_imovel'''
md = smf.ols(modelo, data = df_train)
reg = md.fit_regularized(method = 'elastic_net' 
                             , refit = True
                             , L1_wt = 1
                             , alpha = 0.05)
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     1207.
Date:                Mon, 13 May 2024   Prob (F-statistic):               0.00
Time:                        22:22:56   Log-Likelihood:                -10288.
No. Observations:                9320   AIC:                         2.059e+04
Df Residuals:                    9316   BIC:                         2.062e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      8.1013      0.034    238.106      0.000       8.035       8.168
tempo_emprego                  0.0607      0.001     50.968      0.000       0.058       0.063
sexo                          -0.7848      0.016    -49.226      0.000      -0.816      -0.754
tipo_renda_Empresário               0          0        nan        nan           0           0
idade                          0.0048      0.001      5.579      0.000       0.003       0.006
educacao_Superior_completo          0          0        nan        nan           0           0
qt_pessoas_residencia               0          0        nan        nan           0           0
posse_de_imovel                     0          0        nan        nan           0           0
==============================================================================
Omnibus:                        0.307   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.858   Jarque-Bera (JB):                0.273
Skew:                           0.004   Prob(JB):                        0.872
Kurtosis:                       3.025   Cond. No.                         262.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [43]:
regr= DecisionTreeRegressor(max_depth=8)

regr.fit(X_train, y_train)

r2= regr.score(X_train, y_train)
print("R2 é ",r2)

R2 é  0.7114728249245632
